In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
import sys
import lightgbm

pd.set_option("display.max.columns", None)

/anaconda3/envs/fraud/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
def reduce_mem_usage(data_df, sparse=False):
    """Reduce memory usage of Pandas DF.
    From https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
    
    :param df: Pandas DF
    :type df: :class:`pandas.DataFrame`
    :return: Pandas DF
    :rtype: :class:`pandas.DataFrame`
    """
    megabyte = 1024**2
    start_mem = data_df.memory_usage().sum()/megabyte
    print(f'Memory usage of dataframe is {start_mem:.2f} MB')

    for col in data_df.columns:
        col_type = data_df[col].dtype

        if col_type != object:
            c_min = data_df[col].min()
            c_max = data_df[col].max()

            if str(col_type).startswith('int') or str(col_type).startswith('uint'):
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    data_df[col] = data_df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    data_df[col] = data_df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    data_df[col] = data_df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    data_df[col] = data_df[col].astype(np.int64)
            elif str(col_type).startswith('float'):
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    data_df[col] = data_df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    data_df[col] = data_df[col].astype(np.float32)
                else:
                    data_df[col] = data_df[col].astype(np.float64)
        else:
            data_df[col] = data_df[col].astype('category')

    if sparse:
        data_df = csr_matrix(data_df)
        end_mem = (data_df.data.nbytes + data_df.indptr.nbytes + data_df.indices.nbytes)/megabyte
    else:
        end_mem = data_df.memory_usage().sum()/megabyte
        
    percent_change = 100*(start_mem-end_mem)/start_mem
    print(f'Memory usage after optimization is: {end_mem:.2f} MB')
    print(f'Decreased by {percent_change:.1f}%')

    return data_df

In [3]:
def explode_categoricals(data_df, keep_na=True):
    """
    Take a dataframe, find the categorical data stored as strings, and explode those columns into dummy variables
    
    :param data_df: Input dataframe to examine for categorical data
    :type data_df: :class:`pandas.DataFrame`
    :return: Input dataframe with categorical data represented as dummy variables
    :rype: :class:`pandas.DataFrame`
    """
    
    # Identify categorical columns not already binarized
    string_columns = []
    for column in data_df.columns:
        if data_df[column].dtype=="O":
            string_columns.append(column)

    # Create dummy variables for categorical columns
    for c in string_columns:
        dummy_df = pd.get_dummies(data_df[c], prefix=c, dummy_na=keep_na)
        data_df = pd.concat([data_df, dummy_df], axis=1).drop(c, axis=1)
    
    return data_df

In [4]:
def get_email_region(text_entry):
    if str(text_entry)=="nan":
        return "nan"
    else:
        return str(text_entry).split(".")[-1]

In [5]:
def get_email_site(text_entry):
    if str(text_entry)=="nan":
        return "nan"
    else:
        return str(text_entry).split(".")[0]

In [6]:
input_path = sys.argv[1]
output_path = sys.argv[2]
train_id = pd.read_csv(input_path)
train_id = explode_categoricals(train_id)

In [7]:
train_transaction = pd.read_csv("./data/train_transaction.csv")

train_transaction["P_email_region"] = train_transaction["P_emaildomain"].apply(get_email_region)
train_transaction["P_email_site"] = train_transaction["P_emaildomain"].apply(get_email_site)

train_transaction["R_email_region"] = train_transaction["R_emaildomain"].apply(get_email_region)
train_transaction["R_email_site"] = train_transaction["R_emaildomain"].apply(get_email_site)

train_transaction = explode_categoricals(train_transaction.drop(["P_emaildomain", "R_emaildomain"], axis=1), keep_na=False)

In [8]:
train_df = train_id.merge(right=train_transaction, on="TransactionID", how="inner")

In [14]:
train_df.to_csv(output_path)